In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten,MaxPool2D,Dense,Conv2D,Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os
import cv2
from cv2 import cvtColor
from keras.callbacks import History 


In [ ]:
X=np.zeros((1,64,64,1),dtype=float)  #images
Y=np.zeros((1,10),dtype=float)       #labels

In [ ]:
def extract_data(path):
    X=np.zeros((1,64,64,1),dtype=float)  #images
    Y=np.zeros((1,10),dtype=float)       #labels
    for i in os.listdir(path):
        for j in os.listdir(path+'/'+f'{i}'):
            img=plt.imread(path+'/'+f'{i}/{j}')
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            gray=gray/255
            gray = cv2.resize(gray, (64,64))
            gray=gray.reshape(1,64,64,1)
            X = np.vstack((X,gray))
            y = np.zeros((1,10))
            y[0,int(i)] = 1
            Y = np.vstack((Y,y))
    return X,Y

In [ ]:
def preprocess(X,Y):
    X=X[1:,:,:,:]
    Y=Y[1:,:]
    X,Y=shuffle(X,Y)
    xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2,random_state=40)
    return xtrain,xtest,ytrain,ytest

In [ ]:
def visualize(index):
    plt.imshow(X[index].reshape(64,64))
    plt.show()
    print("The label is",Y[index].argmax(axis=0))

In [ ]:
def train_model(xtrain,xtest,ytrain,ytest):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, padding="same",  input_shape=(64, 64, 1), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Conv2D(64, kernel_size=3, padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=512, activation="relu"))
    model.add(Dropout(0.4))
    model.add(Dense(units=128, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(units=10, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
    history=model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=24,batch_size=64,verbose=1)
    return history,model

In [ ]:
def visualize_model(history):
    plt.figure(figsize=(24,8))
    plt.subplot(1,2,1)
    plt.plot(history.history["val_acc"],label="validation_accuracy",c="red",linewidth=4)
    plt.plot(history.history["acc"],label="training_accuracy",c="green",linewidth=4)
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1,2,2)
    plt.plot(history.history["val_loss"],label="validation loss",c="red",linewidth=4)
    plt.plot(history.history["loss"],label="training_loss",c="green",linewidth=4)
    plt.legend()
    plt.grid(True)
    plt.suptitle("ACC / LOSS",fontsize=18)

    plt.show()

In [ ]:
path='./opencv/sign/dataset6'
index=700
X,Y=extract_data(path)


In [ ]:
visualize(700)


In [ ]:
xtrain,xtest,ytrain,ytest=preprocess(X,Y)


In [ ]:
history,model=train_model(xtrain,xtest,ytrain,ytest)
model.summary()


In [ ]:
visualize_model(history)
model.save("mod7.h5")